In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data_path = '/content/drive/My Drive/ANN_Project_data/'


In [ ]:
import pandas as pd

df = pd.read_csv(data_path + 'filtered_dataset_clean.csv')
df.head()


,ID,url,text,images,top_img,authors,label,word_count,domain,tokens
0,1,http://www.eonline.com/news/901683/janet-jacks...,janet jackson and jermaine dupri are hooking u...,https://akns-images.eonline.com/eol_images/Ent...,https://akns-images.eonline.com/eol_images/Ent...,Unknown,0,119,No Domain,"['janet', 'jackson', 'and', 'jermaine', 'dupri..."
1,2,http://www.yourtango.com/2018311123/7-cringey-...,katy perry and orlando bloom are a celeb coupl...,http://www.yourtango.com/sites/default/files/2...,https://www.yourtango.com/sites/default/files/...,Unknown,0,1205,No Domain,"['katy', 'perry', 'and', 'orlando', 'bloom', '..."
2,3,http://pagesix.com/2018/07/05/vanessa-kirby-sh...,two years into a relationship with her boyfrie...,https://pagesix.com/wp-content/uploads/sites/3...,https://pagesix.com/wp-content/uploads/sites/3...,Unknown,0,230,No Domain,"['two', 'years', 'into', 'a', 'relationship', ..."
3,4,http://www.inquisitr.com/4780146/miranda-lambe...,it s been more than two years since miranda la...,https://www.inquisitr.com/wp-content/uploads/2...,https://www.inquisitr.com/wp-content/uploads/2...,News Writer,0,380,No Domain,"['it', 's', 'been', 'more', 'than', 'two', 'ye..."
4,5,http://toofab.com/2017/11/19/selena-gomez-debu...,getty selena gomez showed up to the american m...,https://images.toofab.com/image/ae/5by6/2025/0...,https://images.toofab.com/image/22/16by9/2017/...,Toofab Staff,0,396,No Domain,"['getty', 'selena', 'gomez', 'showed', 'up', '..."


In [ ]:
import zipfile

with zipfile.ZipFile(data_path + 'preprocessed_images.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/images')


In [ ]:
!pip install transformers


In [ ]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm
import torch

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to("cuda" if torch.cuda.is_available() else "cpu")

base_path = '/content/images'
labels = ['label_0', 'label_1']

results = []

for label_str in labels:
    label_num = int(label_str.split('_')[1])
    folder = os.path.join(base_path, label_str)
    image_files = [f for f in os.listdir(folder) if f.endswith('.jpg')]

    print(f"🔍 Preprocessing {len(image_files)} images from {label_str} ...")

    for filename in tqdm(image_files):
        try:
            image_path = os.path.join(folder, filename)
            raw_image = Image.open(image_path).convert('RGB')
            inputs = processor(raw_image, return_tensors="pt").to(model.device)
            out = model.generate(**inputs)
            caption = processor.decode(out[0], skip_special_tokens=True)
        except Exception as e:
            caption = "ERROR"
            print(f"ERROR: {filename} — {e}")

        results.append({
            'image_name': filename,
            'caption': caption,
            'label': label_num
        })

df = pd.DataFrame(results)

csv_output = '/content/captions_with_labels.csv'
df.to_csv(csv_output, index=False)
print(f"done Saved{csv_output}")


In [ ]:
import os
import shutil

output_dir = '/content/drive/My Drive/ANN_Output'
os.makedirs(output_dir, exist_ok=True)

local_file = '/content/captions_with_labels.csv'
destination = os.path.join(output_dir, 'captions_with_labels.csv')

shutil.copy(local_file, destination)
print(f"Done Saved: {destination}")


✅ تم حفظ الملف في: /content/drive/My Drive/ANN_Output/captions_with_labels.csv


In [ ]:
import zipfile

glove_zip_path = '/content/drive/My Drive/ANN/glove.840B.300d.txt.zip'
we_zip_path = '/content/drive/My Drive/ANN/We.npy.zip'

glove_extract_path = '/content/drive/My Drive/ANN/glove.840B.300d.txt'
we_extract_path = '/content/drive/My Drive/ANN/We.npy'

with zipfile.ZipFile(glove_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')

with zipfile.ZipFile(we_zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')



✅ تم فك ضغط الملفات بنجاح!


In [ ]:
import numpy as np

def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            try:
                coefs = np.asarray(values[1:], dtype='float32')
            except ValueError:
                continue
            embeddings_index[word] = coefs
    return embeddings_index

glove_file = '/content/glove.840B.300d.txt'

glove_embeddings = load_glove_embeddings(glove_file)




In [ ]:
import numpy as np

captions_embeddings = np.load('/content/captions_embeddings.npy')

print(captions_embeddings.shape)

print(captions_embeddings[:5])


In [ ]:
def text_to_embedding(text, glove_embeddings):
    words = text.split()
    embeddings = []
    for word in words:
        embedding = glove_embeddings.get(word.lower())
        if embedding is not None:
            embeddings.append(embedding)
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(300)
captions_embeddings = df['caption'].apply(lambda x: text_to_embedding(str(x), glove_embeddings))

np.save('/content/captions_embeddings.npy', np.array(captions_embeddings.tolist()))



In [ ]:
import pandas as pd

# Load the datasets
filtered_dataset_path = '/content/filtered_dataset_clean.csv'
captions_file_path = '/content/captions_with_labels.csv'

# Load the datasets into pandas DataFrames
filtered_dataset_df = pd.read_csv(filtered_dataset_path)
captions_df = pd.read_csv(captions_file_path)

print(captions_df['image_name'].head())


0    image_1542.jpg
1    image_2179.jpg
2    image_1083.jpg
3     image_370.jpg
4    image_2572.jpg
Name: image_name, dtype: object


In [ ]:
import pandas as pd

# Load the dataset
captions_file_path = '/content/captions_with_labels.csv'
captions_df = pd.read_csv(captions_file_path)

# Check the first few rows to see the structure
print(captions_df.head())

# Drop rows where 'image_name' is missing
captions_df = captions_df.dropna(subset=['image_name'])

# Ensure 'image_name' column contains valid values (split by '_')
captions_df = captions_df[captions_df['image_name'].str.contains('_', na=False)]

# Extract image_number from "image_28.jpg" → 28
captions_df['image_name'] = captions_df['image_name'].str.split('_').str[1].str.split('.').str[0].astype(int)

# Check the result
print(captions_df.head())

# Proceed with merging if necessary


       image_name                                            caption  label
0  image_1542.jpg  khla and khla at the launch party for their ne...      0
1  image_2179.jpg                the girls are dancing in the street      0
2  image_1083.jpg        a man in a red shirt is standing on a stage      0
3   image_370.jpg  a man in a white suit and hat standing on a re...      0
4  image_2572.jpg  a woman in a black shirt and red pants is sitt...      0
   image_name                                            caption  label
0        1542  khla and khla at the launch party for their ne...      0
1        2179                the girls are dancing in the street      0
2        1083        a man in a red shirt is standing on a stage      0
3         370  a man in a white suit and hat standing on a re...      0
4        2572  a woman in a black shirt and red pants is sitt...      0
